In [1]:
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from tqdm import tqdm
import itertools
from collections import Counter
import numpy as np
from sklearn.metrics import precision_score, recall_score, classification_report, average_precision_score, roc_auc_score
import matplotlib.pyplot as plt
from hs_tree_blenders_creation.batch_runners.data_sets import get_account_test_set
import datetime
import pandas as pd

In [2]:
mongo = get_mongo_client_db('production', 'essex')

In [19]:
reqs_filter = {
   # 'req_status': {'$nin': ['Filled', 'Canceled', 'Cancelled', 'Cancel', 'Deleted', 'Requisition rejected', 'Closed', 'close', 'SAPClosed', 'SAPDeleted', 'Requisition request cancelled', 'Cancelled - Changed Emp. Type', 'Declined', 'Denied', 'Event Completed', 'unpublish', 'Approval Denied']}, 
   # 'is_enabled': False, 
  #  'open_date': {'$gte': datetime.datetime(2022, 12, 20, 5, 37, 1, 953162)}
}
reqs = list(mongo.req.find(reqs_filter, []))
len(reqs)

255

In [20]:
req_ids = [r['_id'] for r in reqs]

In [21]:
mongo.recommended_candidate.count({'req_id': {'$in': req_ids}})

/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_94093/2677524474.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  mongo.recommended_candidate.count({'req_id': {'$in': req_ids}})


2762

In [24]:
mongo.recommended_candidate.count({'external_pool': {'$exists': True}})

/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_94093/2428192581.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  mongo.recommended_candidate.count({'external_pool': {'$exists': True}})


0

In [6]:
recs = list(mongo.recommended_candidate.find({'req_id': {'$in': req_ids}}))
len(recs)

2023-03-22T12:31:25.947180Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=14425.966 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=94093 request_id=1954899097 source=PyMongo stack_snippet=
  File "/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_94093/919133386.py", line 1, in <module>
    recs = list(mongo.recommended_candidate.find({'req_id': {'$in': req_ids}}))

2023-03-22T12:31:45.667218Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=19718.83 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log 

1684

In [7]:
df = pd.DataFrame(recs)

In [8]:
fields = set()
for r in recs:
    for k in r.keys():
        if 'external' in k:
            fields.add(k)
fields

{'external_req_id', 'external_source'}

In [10]:
df['pool'] = 'PAST'#df['external_pool'].replace(np.nan, 0).apply(lambda x: x.get('external_pool_id', 'PAST') if x != 0 else 'PAST')

In [11]:
df['pool'].fillna({}).value_counts(normalize=True, dropna=False)

PAST    1.0
Name: pool, dtype: float64

In [12]:
len(df[df['pool'] == 'PAST']['req_id'].unique())

75

In [13]:
len(df[df['pool'] == 'randstad']['req_id'].unique())

0

In [27]:
df.groupby('pool').apply(lambda x: x.groupby('req_id')['_id'].count().describe())

_id,count,mean,std,min,25%,50%,75%,max
pool,,,,,,,,
PAST,75.0,22.453333,11.67614,11.0,15.0,16.0,28.5,82.0


In [25]:
df.groupby('req_id')['_id'].count().describe()

count    75.000000
mean     22.453333
std      11.676140
min      11.000000
25%      15.000000
50%      16.000000
75%      28.500000
max      82.000000
Name: _id, dtype: float64

In [26]:
df.to_csv('essex_recommended_candidates.csv')